In [1]:
import numpy as np

from elastica.modules import BaseSystemCollection, Constraints, Forcing, CallBacks, Damping
from elastica.rod.cosserat_rod3 import CosseratRod
from elastica.dissipation import AnalyticalLinearDamper
from elastica.external_forces import GravityForces, MuscleTorques, EndpointForces
from elastica.interaction import AnisotropicFrictionalPlane
from elastica.boundary_conditions import OneEndFixedBC

from elastica.timestepper.symplectic_steppers import PositionVerlet
from elastica.timestepper import integrate

from elastica.callback_functions import CallBackBaseClass
from collections import defaultdict

from IPython.display import Video
from tqdm import tqdm

print("imports done")



class OctoArmSimulator(BaseSystemCollection, Constraints, Forcing, CallBacks, Damping):
    pass

octopus_sim = OctoArmSimulator()

n_elem = 50
start = np.array([0.0, 0.0, 0.0])
direction = np.array([0.0, 0.0, 1.0])
normal = np.array([0.0, 1.0, 0.0])
base_length = 0.2 #0.35
base_radius = 0.01 #base_length * 0.011
tip_radius = base_radius*0.1 #base_length*0.005 #added this
base_area = np.pi * base_radius ** 2
density = 1042 #1050 #1000
nu = 2e-1 #2e-3
E = 1e4 #10 #1e6 
poisson_ratio = 0.5
shear_modulus = E / (2*(poisson_ratio + 1.0))

shearable_rod = CosseratRod.straight_rod(
    n_elem,
    start,
    direction,
    normal,
    base_length,
    base_radius,
    density,
    youngs_modulus=E,
    shear_modulus=shear_modulus,
    tip_radius=tip_radius #added this
)

octopus_sim.append(shearable_rod)

dt = 1e-5
octopus_sim.dampen(shearable_rod).using(
    AnalyticalLinearDamper,
    damping_constant=nu,
    time_step=dt,
)

gravitational_acc = -9.80665
octopus_sim.add_forcing_to(shearable_rod).using(
    GravityForces, acc_gravity=np.array([0.0, gravitational_acc, 0.0])
)
print("Gravity now acting on octopus arm")

period = 2.0
wave_length = 4.0 #1
b_coeff = np.array([3.4e-3, 3.3e-3, 4.2e-3, 2.6e-3, 3.6e-3, 3.5e-3])

octopus_sim.add_forcing_to(shearable_rod).using(
    MuscleTorques,
    base_length=base_length,
    b_coeff=b_coeff,
    period=period,
    wave_number=2.0 * np.pi / (wave_length),
    phase_shift=0.0,
    rest_lengths=shearable_rod.rest_lengths,
    ramp_up_time=period,
    direction=normal,
    with_spline=True,
)
print("Muscle torques added to the octopus arm")

origin_plane = np.array([0.0, -base_radius, 0.0])
normal_plane = normal
slip_velocity_tol = 1e-8
froude = 0.1
mu = base_length / (period * period * np.abs(gravitational_acc) * froude)
kinetic_mu_array = np.array(
    [1.0 * mu, 1.5 * mu, 2.0 * mu] 
)  # [forward, backward, sideways]
static_mu_array = 2 * kinetic_mu_array


octopus_sim.add_forcing_to(shearable_rod).using(
    AnisotropicFrictionalPlane,
    k=1.0,
    nu=1e-6,
    plane_origin=origin_plane,
    plane_normal=normal_plane,
    slip_velocity_tol=slip_velocity_tol,
    static_mu_array=static_mu_array,
    kinetic_mu_array=kinetic_mu_array,
)
print("Friction forces added to the octopus arm")


octopus_sim.constrain(shearable_rod).using(
    OneEndFixedBC, constrained_position_idx=(0,), constrained_director_idx=(0,)
)


start_force_x = 1e-1 #change to whatever
start_force = np.array([0.0, 0.0, start_force_x])
octopus_sim.add_forcing_to(shearable_rod).using(
    EndpointForces, 
    0.0*start_force, 
    start_force, #this is the slot for end_force, and we want it to be 0, unless...?
    ramp_up_time = 1e-1
    )

class OctopusArmCallBack(CallBackBaseClass):
    """
    Call back function for continuum octopus arm
    """

    def __init__(self, step_skip: int, callback_params: dict):
        CallBackBaseClass.__init__(self)
        self.every = step_skip
        self.callback_params = callback_params

    def make_callback(self, system, time, current_step: int):

        if current_step % self.every == 0:

            self.callback_params["time"].append(time)
            self.callback_params["step"].append(current_step)
            self.callback_params["position"].append(system.position_collection.copy())
            #print(self.callback_params["position"])
            self.callback_params["velocity"].append(system.velocity_collection.copy())
            self.callback_params["avg_velocity"].append(
                system.compute_velocity_center_of_mass()
            )

            self.callback_params["center_of_mass"].append(
                system.compute_position_center_of_mass()
            )
            self.callback_params["curvature"].append(system.kappa.copy())

            return


pp_list = defaultdict(list)
octopus_sim.collect_diagnostics(shearable_rod).using(
    OctopusArmCallBack, step_skip=100, callback_params=pp_list
)
print("Callback function added to the simulator")

octopus_sim.finalize()

final_time = 2.5 * period #5
total_steps = int(final_time / dt)
#print("Total steps", total_steps)

timestepper = PositionVerlet()

integrate(timestepper, octopus_sim, final_time, total_steps)
#print(pp_list["position"]) #to check 

def plot_video_2D(plot_params: dict, video_name="video.mp4", margin=0.2, fps=15):
    !pip install matplotlib==3.7.5
    from matplotlib import pyplot as plt
    import matplotlib.animation as manimation

    t = np.array(plot_params["time"])
    positions_over_time = np.array(plot_params["position"])
    total_time = int(np.around(t[..., -1], 1))
    total_frames = fps * total_time
    step = round(len(t) / total_frames)
    
    print("creating video -- this can take a few minutes")
    FFMpegWriter = manimation.writers["ffmpeg"]
    metadata = dict(title="Movie Test", artist="Matplotlib", comment="Movie support!")
    writer = FFMpegWriter(fps=fps, metadata=metadata)

    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.axis("equal")
    rod_lines_2d = ax.plot(
        positions_over_time[0][2], positions_over_time[0][0], linewidth=3
    )[0]
    ax.set_xlim([0 - margin, 1 + margin])
    ax.set_ylim([-1.5 - margin, 1.5 + margin])
    with writer.saving(fig, video_name, dpi=100):
        with plt.style.context("default"):
            for i,time in enumerate(range(1, len(t), step)):
                rod_lines_2d.set_xdata(positions_over_time[time][2])
                rod_lines_2d.set_ydata(positions_over_time[time][0])

                writer.grab_frame()

    plt.close(fig)


filename_video = "octopus_arm.mp4"
plot_video_2D(pp_list, video_name=filename_video, margin=0.2, fps=125)

Video("octopus_arm.mp4")


imports done
Gravity now acting on octopus arm
Muscle torques added to the octopus arm
Friction forces added to the octopus arm
Callback function added to the simulator


100%|████████████████████████████████████████████████████████████████████████| 499999/499999 [01:03<00:00, 7866.07it/s]

Final time of simulation is :  4.999999999880011


creating video -- this can take a few minutes


In [2]:
from elastica.rod.factory_function2 import allocate
import numpy as np

n_elem = 50
start = np.array([0.0, 0.0, 0.0])
direction = np.array([0.0, 0.0, 1.0])
normal = np.array([0.0, 1.0, 0.0])
base_length = 0.35 #0.2 #0.35
base_radius = base_length * 0.011 #0.01 #base_length * 0.011
tip_radius = base_radius*0.1 #base_length*0.005 #added this
base_area = np.pi * base_radius ** 2
density = 1000 #1050 #1000
nu = 2e-3 #2e-3
E = 1e6 #10 #1e6 
poisson_ratio = 0.5
shear_modulus = E / (1*(poisson_ratio + 1.0))


print(allocate(
            n_elem,
            direction,
            normal,
            base_length,
            base_radius,
            density,
            youngs_modulus=E,
            rod_origin_position=start,
            ring_rod_flag=False,
            shear_modulus=shear_modulus, #added cuz it was gone for some reason
            tip_radius=tip_radius, #added this
            #**kwargs,
        ))

(50, array([[0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.007, 0.014, 0.021, 0.028, 0.035, 0.042, 0.049, 0.056,
        0.063, 0.07 , 0.077, 0.084, 0.091, 0.098, 0.105, 0.112, 0.119,
        0.126, 0.133, 0.14 , 0.147, 0.154

In [3]:
from elastica.rod.factory_function import allocate
import numpy as np

n_elem = 50
start = np.array([0.0, 0.0, 0.0])
direction = np.array([0.0, 0.0, 1.0])
normal = np.array([0.0, 1.0, 0.0])
base_length = 0.35 #0.2 #0.35
base_radius = base_length * 0.011 #0.01 #base_length * 0.011
tip_radius = 0.35 #base_length*0.005 #added this
base_area = np.pi * base_radius ** 2
density = 1000 #1050 #1000
nu = 2e-3 #2e-3
E = 1e6 #10 #1e6 
poisson_ratio = 0.5
shear_modulus = E / (1*(poisson_ratio + 1.0))


print(allocate(
            n_elem,
            direction,
            normal,
            base_length,
            base_radius,
            density,
            youngs_modulus=E,
            rod_origin_position=start,
            ring_rod_flag=False,
            shear_modulus=shear_modulus, #added cuz it was gone for some reason
            #tip_radius=tip_radius, #added this
            #**kwargs,
        ))

(50, array([[0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.007, 0.014, 0.021, 0.028, 0.035, 0.042, 0.049, 0.056,
        0.063, 0.07 , 0.077, 0.084, 0.091, 0.098, 0.105, 0.112, 0.119,
        0.126, 0.133, 0.14 , 0.147, 0.154

In [4]:
from elastica.rod.factory_function2 import allocate
from elastica.rod.cosserat_rod2 import CosseratRod
import numpy as np

n_elem = 50
start = np.array([0.0, 0.0, 0.0])
direction = np.array([0.0, 0.0, 1.0])
normal = np.array([0.0, 1.0, 0.0])
base_length = 0.35 #0.2 #0.35
base_radius = base_length * 0.011 #0.01 #base_length * 0.011
tip_radius = base_radius*0.1 #base_length*0.005 #added this
base_area = np.pi * base_radius ** 2
density = 1000 #1050 #1000
nu = 2e-3 #2e-3
E = 1e6 #10 #1e6 
poisson_ratio = 0.5
shear_modulus = E / (2*(poisson_ratio + 1.0))

"""
print(allocate(
            n_elem,
            direction,
            normal,
            base_length,
            base_radius,
            density,
            youngs_modulus=E,
            rod_origin_position=start,
            ring_rod_flag=False,
            shear_modulus=shear_modulus, #added cuz it was gone for some reason
            tip_radius=tip_radius, #added this
            #**kwargs,
        ))
"""
octo_arm = CosseratRod.straight_rod(
    n_elem,
    start,
    direction,
    normal,
    base_length,
    base_radius,
    density,
    youngs_modulus=E,
    shear_modulus=shear_modulus,
    tip_radius=tip_radius #added this
)

print(octo_arm.radius)

[0.00385    0.00377929 0.00370857 0.00363786 0.00356714 0.00349643
 0.00342571 0.003355   0.00328429 0.00321357 0.00314286 0.00307214
 0.00300143 0.00293071 0.00286    0.00278929 0.00271857 0.00264786
 0.00257714 0.00250643 0.00243571 0.002365   0.00229429 0.00222357
 0.00215286 0.00208214 0.00201143 0.00194071 0.00187    0.00179929
 0.00172857 0.00165786 0.00158714 0.00151643 0.00144571 0.001375
 0.00130429 0.00123357 0.00116286 0.00109214 0.00102143 0.00095071
 0.00088    0.00080929 0.00073857 0.00066786 0.00059714 0.00052643
 0.00045571 0.000385  ]
